In [0]:
bronze_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/bronze/"
silver_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/silver/"
fold_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/gold/"
resource_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/resource/origem"
resource_path_volume = "/Volumes/bikestore/logistica/bikestore_resource/origem/"

In [0]:
display(dbutils.fs.ls(bronze_path))

In [0]:
display(dbutils.fs.ls(resource_path))

In [0]:
%sql
create table if not exists bikestore.logistica.bronze_brands
location "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/bronze/brands/"

In [0]:
(spark.read.format("delta")
 .load(f"{bronze_path}/customers/")
 .createOrReplaceTempView("tmp_customers"))

In [0]:
%sql
select * from tmp_customers

In [0]:
bronze_map= {
    "tmp_bronze_brands": f"{bronze_path}/brands/",
    "tmp_bronze_categories": f"{bronze_path}/categories/",
    "tmp_bronze_customers": f"{bronze_path}/customers/",
    "tmp_bronze_order_items": f"{bronze_path}/order_items/",
    "tmp_bronze_orders": f"{bronze_path}/orders/",
    "tmp_bronze_products": f"{bronze_path}/products/",
    "tmp_bronze_staffs": f"{bronze_path}/staffs/",
    "tmp_bronze_stocks": f"{bronze_path}/stocks/",
    "tmp_bronze_stores": f"{bronze_path}/stores/",
}
for view_name, path in bronze_map.items():
    (spark.read.format("delta")
     .load(path)
     .createOrReplaceTempView(view_name))

In [0]:
%sql
show tables in bikestore.logistica

In [0]:
%sql
select * from tmp_bronze_stores

Criando uma cada Silver de teste

In [0]:
%sql
select * from tmp_bronze_products limit 10

In [0]:
%sql
select * from tmp_bronze_categories limit 10

In [0]:
%sql
describe tmp_bronze_categories

In [0]:
%sql
select
  p.product_id,
  p.product_name,
  p.brand_id,
 c.category_name,
 -- c.category_id as category_id_categoriat,
 -- p.category_id,
  p.model_year,
  p.list_price
from tmp_bronze_products as p
left join tmp_bronze_categories as c on p.category_id = c.category_id



In [0]:
df_prodcut_silver= spark.sql("""
    select
    p.product_id,
    p.product_name,
    p.brand_id,
    c.category_name,
    -- c.category_id as category_id_categoriat,
    -- p.category_id,
    p.model_year,
    p.list_price
    from tmp_bronze_products as p
    left join tmp_bronze_categories as c on p.category_id = c.category_id""")

display(df_prodcut_silver)

In [0]:
df_prodcut_silver.write.mode("overwrite").format("delta").option("merfeSchema", "true").save(f"{silver_path}product_teste/")

In [0]:
%sql
create table if not exists bikestore.logistica.product_teste
location "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/silver/product_teste"

In [0]:
%sql
select * from bikestore.logistica.product_teste